In [1]:
import sympy as sp
import numpy as np
from sympy import sin, cos, atan2, diag, Matrix, zeros, simplify, diff
from sympy.codegen.ast import CodeBlock, Assignment

# Start timing
import time
start_time = time.time()

# Define symbolic variables
h, r, m, g, h2 = sp.symbols('h r m g h2')  # height, arm radius, arm mass fraction, gravity, arm height
yaw, pit, rol, q4 = sp.symbols('yaw pit rol q4')  # coordinates (rad)
yawd, pitd, rold, q4d = sp.symbols('yawd pitd rold q4d')  # velocities (rad/s)
Isx, Isy, Isz = sp.symbols('Isx Isy Isz')  # shell inertia
Iax, Iay, Iaz = sp.symbols('Iax Iay Iaz')  # arm inertia

Is = diag(Isx, Isy, Isz)  # shell inertia tensor
Ia = diag(Iax, Iay, Iaz)  # arm inertia tensor

q = Matrix([yaw, pit, rol, q4])  # coordinate vector
qd = Matrix([yawd, pitd, rold, q4d])  # velocity vector

# Forward kinematics
R01 = Matrix([[cos(yaw), -sin(yaw), 0],
              [sin(yaw), cos(yaw), 0],
              [0, 0, 1]])
d01 = Matrix([0, 0, 0])

R12 = Matrix([[cos(pit), 0, sin(pit)],
              [0, 1, 0],
              [-sin(pit), 0, cos(pit)]])
d12 = Matrix([0, 0, 0])

R23 = Matrix([[1, 0, 0],
              [0, cos(rol), -sin(rol)],
              [0, sin(rol), cos(rol)]])
d23 = Matrix([0, -h * sin(rol), h * cos(rol)])

R34 = Matrix([[cos(q4), -sin(q4), 0],
              [sin(q4), cos(q4), 0],
              [0, 0, 1]])
d34 = Matrix([r * cos(q4), r * sin(q4), h2])

# Homogeneous transformation matrices
H01 = R01.row_join(d01).col_join(Matrix([0, 0, 0, 1]).T)
H12 = R12.row_join(d12).col_join(Matrix([0, 0, 0, 1]).T)
H23 = R23.row_join(d23).col_join(Matrix([0, 0, 0, 1]).T)
H34 = R34.row_join(d34).col_join(Matrix([0, 0, 0, 1]).T)

R02 = R01 * R12
R03 = R02 * R23
R04 = R03 * R34

H02 = H01 * H12
H03 = H02 * H23
H04 = H03 * H34

CG = H04 * Matrix([-r * 1/(1+m), 0, -h2 * 1/(1+m), 1])  # CG location
pts = [Matrix([0, 0, 0, 1]), H03[:, 3], H03 * Matrix([0.3 * r, 0, 0, 1]),
       H03[:, 3], CG, H04[:, 3]]

# Differential kinematics
def diffT(T, vars, order):
    """Compute time derivative of vector T."""
    n = len(vars) // 2
    q = vars[:n]  # joint positions
    qd = vars[n:]  # joint velocities

    T = Matrix(T)
    q = Matrix(q)
    qd = Matrix(qd)
    
    # Compute Jacobian and first derivative
    J = T.jacobian(q)  # Use Matrix.jacobian method
    dT = J * qd  # dT/dt = ∂T/∂q * dq/dt
    
    # Higher-order derivatives
    if order > 1:
        for _ in range(order - 1):
            dT = diffT(dT, vars, 1)
    return dT

z00 = Matrix([0, 0, 1])
y01 = R01[:, 1]
x02 = R02[:, 0]
z03 = R03[:, 2]

J1v = zeros(3, 4)
J2v = zeros(3, 4)
J3v = Matrix.hstack(z00.cross(H03[:3, 3]),
                    y01.cross(H03[:3, 3]),
                    x02.cross(H03[:3, 3]),
                    zeros(3, 1))
J4v = Matrix.hstack(z00.cross(H04[:3, 3]),
                    y01.cross(H04[:3, 3]),
                    x02.cross(H04[:3, 3]),
                    z03.cross(H04[:3, 3]) - H03[:3, 3])

J1w = Matrix.hstack(z00, zeros(3, 3))
J2w = Matrix.hstack(z00, y01, zeros(3, 2))
J3w = Matrix.hstack(z00, y01, x02, zeros(3, 1))
J4w = Matrix.hstack(z00, y01, x02, z03)

v_CG = diffT(CG[:3], Matrix([q, qd]), 1)
a_CG = diffT(v_CG, Matrix([q, qd]), 1)
F = a_CG + Matrix([0, 0, 9.81])

# Potential energy
P = 1 * g * H03[2, 3] + m * g * H04[2, 3]  # Potential energy (J)
G = P.diff(q).T  # Use Matrix.jacobian method

print('G =', G)

# M & C matrices
M = 1 * (J3v.T * J3v) + (J3w.T * R03 * Is * R03.T * J3w) + \
    m * (J4v.T * J4v) + (J4w.T * R04 * Ia * R04.T * J4w)

K = 0.5 * qd.T * M * qd
E = P + K[0]  # K is a 1x1 matrix, extract scalar

C = zeros(4, 4)
for k in range(4):
    for j in range(4):
        print(f"Computing row {k}, column {j} of C")
        for i in range(4):
            C[k, j] += 0.5 * (diff(M[k, j], q[i]) +
                             diff(M[k, i], q[j]) -
                             diff(M[i, j], q[k])) * qd[i]
# C = simplify(C)

# External forces
J_CG = Matrix(CG[:3]).jacobian(q)  # Use Matrix.jacobian method

# Display timing
elapsed_time = time.time() - start_time
print(f"symbolics in {elapsed_time:.2f}s")

G = Matrix([[0, -g*h*sin(pit)*cos(rol) + g*m*(-h*sin(pit)*cos(rol) - h2*sin(pit)*cos(rol) - r*sin(pit)*sin(q4)*sin(rol) - r*cos(pit)*cos(q4)), -g*h*sin(rol)*cos(pit) + g*m*(-h*sin(rol)*cos(pit) - h2*sin(rol)*cos(pit) + r*sin(q4)*cos(pit)*cos(rol)), g*m*(r*sin(pit)*sin(q4) + r*sin(rol)*cos(pit)*cos(q4))]])
Computing row 0, column 0 of C
Computing row 0, column 1 of C
Computing row 0, column 2 of C
Computing row 0, column 3 of C
Computing row 1, column 0 of C
Computing row 1, column 1 of C
Computing row 1, column 2 of C
Computing row 1, column 3 of C
Computing row 2, column 0 of C
Computing row 2, column 1 of C
Computing row 2, column 2 of C
Computing row 2, column 3 of C
Computing row 3, column 0 of C
Computing row 3, column 1 of C
Computing row 3, column 2 of C
Computing row 3, column 3 of C
symbolics in 1.24s


In [20]:
M_simp = zeros(M.shape[0], M.shape[1])
for i in range(M.shape[0]):
    for j in range(M.shape[1]):
        start_time = time.time()
        M_simp[i, j] = simplify(M[i, j])
        print(f"Simplifing M[{i}, {j}] took {time.time() - start_time:.2f}s, reducing ops from {sp.count_ops(M[i,j])} to {sp.count_ops(M_simp[i,j])}")

Simplifing M[0, 0] took 1.86s, reducing ops from 163 to 206
Simplifing M[0, 1] took 3.59s, reducing ops from 400 to 178
Simplifing M[0, 2] took 5.68s, reducing ops from 587 to 92
Simplifing M[0, 3] took 5.83s, reducing ops from 469 to 53
Simplifing M[1, 0] took 1.86s, reducing ops from 361 to 129
Simplifing M[1, 1] took 3.72s, reducing ops from 650 to 93
Simplifing M[1, 2] took 7.47s, reducing ops from 1151 to 28
Simplifing M[1, 3] took 7.11s, reducing ops from 893 to 26
Simplifing M[2, 0] took 3.02s, reducing ops from 567 to 92
Simplifing M[2, 1] took 5.86s, reducing ops from 1114 to 28
Simplifing M[2, 2] took 4.97s, reducing ops from 1219 to 27
Simplifing M[2, 3] took 6.97s, reducing ops from 1117 to 13
Simplifing M[3, 0] took 4.06s, reducing ops from 463 to 45
Simplifing M[3, 1] took 5.41s, reducing ops from 870 to 26
Simplifing M[3, 2] took 6.48s, reducing ops from 1169 to 13
Simplifing M[3, 3] took 4.43s, reducing ops from 949 to 6


In [23]:
M_sym = sp.MatrixSymbol("M", M.shape[0], M.shape[1])

codeblock = CodeBlock(
    Assignment(M_sym, M_simp),
).cse() #common subexpression elimination
num_ops = sp.count_ops(codeblock)
code = sp.ccode(codeblock)

print(f"# {num_ops} operations")
print(code)

# 305 operations
x0 = sin(pit);
x1 = cos(pit);
x2 = pow(x1, 2);
x3 = cos(rol);
x4 = pow(x3, 2);
x5 = Iaz*x4;
x6 = sin(rol);
x7 = pow(x6, 2);
x8 = Isz*x4;
x9 = sin(q4);
x10 = cos(q4);
x11 = x1*x6;
x12 = x0*x9 + x10*x11;
x13 = x0*x10;
x14 = x11*x9;
x15 = x13 - x14;
x16 = pow(h, 2);
x17 = sin(yaw);
x18 = cos(yaw);
x19 = x0*x3;
x20 = x17*x6 + x18*x19;
x21 = x17*x19 - x18*x6;
x22 = pow(h2, 2);
x23 = pow(r, 2);
x24 = 2*h;
x25 = h2*x24;
x26 = x2*x23;
x27 = pow(x10, 2);
x28 = x23*x27;
x29 = x26*x4;
x30 = x2*x4;
x31 = x1*x3;
x32 = r*x13;
x33 = h2*x31;
x34 = x16*x4;
x35 = h*x6;
x36 = 2*x35;
x37 = x3*x9;
x38 = r*x37;
x39 = x2*x38;
x40 = h2*x6;
x41 = 2*x40;
x42 = x11*x3;
x43 = Iaz*x6;
x44 = m*r;
x45 = x1*x9;
x46 = x44*x45;
x47 = h2*x46;
x48 = Iay*x37;
x49 = Iax*x37;
x50 = m*x16;
x51 = m*x22;
x52 = m*x36;
x53 = m*x32;
x54 = pow(x9, 2);
x55 = Iax*x54;
x56 = Iay*x54;
x57 = m*x23;
x58 = x37*x57;
x59 = x54*x57;
x60 = x0*x16;
x61 = m*x25;
x62 = Iay*x10;
x63 = h*x31;
x64 = h2*x10;
x65 = Iax*x0*x54 - Iax*

In [24]:
C_simp = zeros(4, 4)
for k in range(4):
    for j in range(4):
        print(f"Computing row {k}, column {j} of C")
        for i in range(4):
            C_simp[k, j] += 0.5 * (
                diff(M_simp[k, j], q[i]) +
                diff(M_simp[k, i], q[j]) -
                diff(M_simp[i, j], q[k])
            ) * qd[i]

Computing row 0, column 0 of C
Computing row 0, column 1 of C
Computing row 0, column 2 of C
Computing row 0, column 3 of C
Computing row 1, column 0 of C
Computing row 1, column 1 of C
Computing row 1, column 2 of C
Computing row 1, column 3 of C
Computing row 2, column 0 of C
Computing row 2, column 1 of C
Computing row 2, column 2 of C
Computing row 2, column 3 of C
Computing row 3, column 0 of C
Computing row 3, column 1 of C
Computing row 3, column 2 of C
Computing row 3, column 3 of C


In [30]:
C_simp_simp = zeros(C.shape[0], C.shape[1])
for i in range(C.shape[0]):
    for j in range(C.shape[1]):
        start_time = time.time()
        C_simp_simp[i, j] = simplify(C_simp[i, j])
        print(f"Simplifing C[{i}, {j}] took {time.time() - start_time:.2f}s, reducing ops from {sp.count_ops(C_simp[i,j])} to {sp.count_ops(C_simp_simp[i,j])}")
print(sp.count_ops(C), sp.count_ops(C_simp), sp.count_ops(C_simp_simp))

Simplifing C[0, 0] took 12.09s, reducing ops from 752 to 562
Simplifing C[0, 1] took 9.96s, reducing ops from 936 to 741
Simplifing C[0, 2] took 5.63s, reducing ops from 694 to 429
Simplifing C[0, 3] took 4.44s, reducing ops from 511 to 394
Simplifing C[1, 0] took 26.89s, reducing ops from 789 to 593
Simplifing C[1, 1] took 1.68s, reducing ops from 511 to 264
Simplifing C[1, 2] took 5.52s, reducing ops from 581 to 428
Simplifing C[1, 3] took 2.18s, reducing ops from 334 to 212
Simplifing C[2, 0] took 6.52s, reducing ops from 726 to 505
Simplifing C[2, 1] took 4.37s, reducing ops from 531 to 413
Simplifing C[2, 2] took 0.02s, reducing ops from 20 to 9
Simplifing C[2, 3] took 1.63s, reducing ops from 251 to 185
Simplifing C[3, 0] took 3.79s, reducing ops from 458 to 352
Simplifing C[3, 1] took 2.21s, reducing ops from 334 to 215
Simplifing C[3, 2] took 1.49s, reducing ops from 240 to 171
Simplifing C[3, 3] took 0.02s, reducing ops from 83 to 0
222017 7751 5473


In [38]:
G_simp = zeros(G.shape[0], G.shape[1])
for i in range(G.shape[1]):
    start_time = time.time()
    G_simp[i] = simplify(G[i])
    print(f"Simplifing G[{i}] took {time.time() - start_time:.2f}s, reducing ops from {sp.count_ops(G[i])} to {sp.count_ops(G_simp[i])}")
print(sp.count_ops(G), sp.count_ops(G_simp))

Simplifing G[0] took 0.00s, reducing ops from 0 to 0
Simplifing G[1] took 0.16s, reducing ops from 30 to 32
Simplifing G[2] took 0.07s, reducing ops from 24 to 18
Simplifing G[3] took 0.03s, reducing ops from 13 to 12
67 62


In [78]:
M_sym = sp.MatrixSymbol("M", M.shape[0], M.shape[1])
C_sym = sp.MatrixSymbol("C", C.shape[0], C.shape[1])
G_sym = sp.MatrixSymbol("G", G.shape[0], G.shape[1])

codeblock = CodeBlock(
    Assignment(M_sym, M_simp),
    Assignment(C_sym, C_simp_simp),
    Assignment(G_sym, G_simp),
).cse() #common subexpression elimination
num_ops = sp.count_ops(codeblock)
code = sp.ccode(codeblock)

print(f"# {num_ops} operations")
print(code)

# 932 operations
x0 = sin(pit);
x1 = cos(pit);
x2 = pow(x1, 2);
x3 = cos(rol);
x4 = pow(x3, 2);
x5 = Iaz*x4;
x6 = sin(rol);
x7 = pow(x6, 2);
x8 = Isy*x7;
x9 = Isz*x4;
x10 = sin(q4);
x11 = x0*x10;
x12 = cos(q4);
x13 = x1*x12;
x14 = x13*x6;
x15 = x11 + x14;
x16 = x0*x12;
x17 = x1*x10;
x18 = x17*x6;
x19 = x16 - x18;
x20 = pow(h, 2);
x21 = sin(yaw);
x22 = cos(yaw);
x23 = x0*x3;
x24 = x21*x6 + x22*x23;
x25 = x21*x23 - x22*x6;
x26 = pow(h2, 2);
x27 = pow(r, 2);
x28 = h*h2;
x29 = 2*x28;
x30 = x2*x27;
x31 = pow(x12, 2);
x32 = x27*x31;
x33 = x30*x4;
x34 = x2*x4;
x35 = r*x3;
x36 = x1*x35;
x37 = 2*x36;
x38 = h2*x16;
x39 = x20*x4;
x40 = x26*x4;
x41 = 2*x31;
x42 = h*x6;
x43 = x10*x35;
x44 = 2*x2;
x45 = x43*x44;
x46 = h2*x6;
x47 = Iay*x3;
x48 = x1*x6;
x49 = x1*x3;
x50 = x49*x6;
x51 = Iaz*x6;
x52 = m*r;
x53 = x17*x52;
x54 = h*x53;
x55 = h2*x53;
x56 = x10*x47;
x57 = Iax*x3;
x58 = x10*x57;
x59 = m*x20;
x60 = m*x26;
x61 = h2*x49;
x62 = 2*m;
x63 = x42*x62;
x64 = r*x16;
x65 = m*x64;
x66 = x42*x65;
x67 = -

In [88]:
code_rep = code
for n in range(16):
    row = n // 4
    col = n % 4
    code_rep = code_rep.replace(f"[{n}]", f"[{row},{col}]")
code_rep = code_rep.replace(";", "")


print(code_rep)

x0 = sin(pit)
x1 = cos(pit)
x2 = pow(x1, 2)
x3 = cos(rol)
x4 = pow(x3, 2)
x5 = Iaz*x4
x6 = sin(rol)
x7 = pow(x6, 2)
x8 = Isy*x7
x9 = Isz*x4
x10 = sin(q4)
x11 = x0*x10
x12 = cos(q4)
x13 = x1*x12
x14 = x13*x6
x15 = x11 + x14
x16 = x0*x12
x17 = x1*x10
x18 = x17*x6
x19 = x16 - x18
x20 = pow(h, 2)
x21 = sin(yaw)
x22 = cos(yaw)
x23 = x0*x3
x24 = x21*x6 + x22*x23
x25 = x21*x23 - x22*x6
x26 = pow(h2, 2)
x27 = pow(r, 2)
x28 = h*h2
x29 = 2*x28
x30 = x2*x27
x31 = pow(x12, 2)
x32 = x27*x31
x33 = x30*x4
x34 = x2*x4
x35 = r*x3
x36 = x1*x35
x37 = 2*x36
x38 = h2*x16
x39 = x20*x4
x40 = x26*x4
x41 = 2*x31
x42 = h*x6
x43 = x10*x35
x44 = 2*x2
x45 = x43*x44
x46 = h2*x6
x47 = Iay*x3
x48 = x1*x6
x49 = x1*x3
x50 = x49*x6
x51 = Iaz*x6
x52 = m*r
x53 = x17*x52
x54 = h*x53
x55 = h2*x53
x56 = x10*x47
x57 = Iax*x3
x58 = x10*x57
x59 = m*x20
x60 = m*x26
x61 = h2*x49
x62 = 2*m
x63 = x42*x62
x64 = r*x16
x65 = m*x64
x66 = x42*x65
x67 = -x66
x68 = pow(x10, 2)
x69 = Iax*x68
x70 = Iay*x68
x71 = 2*x7
x72 = m*x27
x73 = x10*x

In [72]:
import sympy as sp
import time

def get_minor(matrix, i, j):
    """
    Calculate the minor of matrix by removing row i and column j.
    """
    minor = matrix.copy()
    minor.row_del(i)
    minor.col_del(j)
    return minor

def determinant_3x3(matrix):
    """
    Calculate the determinant of a 3x3 matrix using the cofactor expansion.
    """
    if matrix.shape != (3, 3):
        raise ValueError("Input must be a 3x3 matrix")
    
    det = 0
    for j in range(3):
        cofactor = matrix[0, j] * ((-1) ** (0 + j))
        minor = get_minor(matrix, 0, j)
        minor_det = minor[0, 0] * minor[1, 1] - minor[0, 1] * minor[1, 0]
        det += cofactor * minor_det
    
    return det

def calculate_determinant_4x4_with_timing(matrix):
    """
    Calculate the determinant of a 4x4 matrix manually with timing information.
    """
    if matrix.shape != (4, 4):
        raise ValueError("Input must be a 4x4 matrix")
    
    start_total = time.time()
    
    # Calculate determinant using cofactor expansion along the first row
    det = 0
    cofactor_times = []
    
    print("Computing 4x4 determinant:")
    
    for j in range(4):
        start_cofactor = time.time()
        
        # Get the element in the first row
        element = matrix[0, j]
        
        # Calculate sign for cofactor
        sign = (-1) ** (0 + j)
        
        # Get the minor by removing first row and column j
        minor = get_minor(matrix, 0, j)
        
        # Calculate determinant of the 3x3 minor
        start_minor = time.time()
        minor_det = determinant_3x3(minor)
        end_minor = time.time()
        
        # Calculate contribution to determinant
        contribution = element * sign * minor_det
        det += contribution
        
        end_cofactor = time.time()
        cofactor_time = end_cofactor - start_cofactor
        
        cofactor_times.append(cofactor_time)
    
    end_total = time.time()
    total_time = end_total - start_total
    
    print(f"  Total determinant: {det}")
    print(f"  Total calculation time: {total_time:.6f} seconds")
    
    return det

def calculate_inverse_4x4_with_timing(matrix):
    """
    Calculate the inverse of a 4x4 matrix using the adjugate method with timing information.
    """
    if matrix.shape != (4, 4):
        raise ValueError("Input must be a 4x4 matrix")
    
    start_total = time.time()
    
    # Calculate determinant first
    print("Step 1: Calculate the determinant")
    start_det = time.time()
    det = calculate_determinant_4x4_with_timing(matrix)
    end_det = time.time()
    det_time = end_det - start_det
    
    if det == 0:
        print("\nMatrix is singular (determinant = 0). Inverse does not exist.")
        return None
    
    print(f"\nStep 2: Calculate the cofactor matrix (and adjugate)")
    
    # Calculate the cofactor matrix
    start_cofactor_matrix = time.time()
    cofactor_matrix = sp.zeros(4, 4)
    cofactor_times = []
    
    for i in range(4):
        for j in range(4):
            start_cofactor = time.time()
            
            # Get the minor by removing row i and column j
            minor = get_minor(matrix, i, j)
            
            # Calculate determinant of the 3x3 minor
            minor_det = determinant_3x3(minor)
            
            # Calculate the cofactor (include the sign)
            sign = (-1) ** (i + j)
            cofactor = sign * minor_det
            
            # Store the cofactor in the cofactor matrix
            cofactor_matrix[i, j] = cofactor
            
            end_cofactor = time.time()
            cofactor_time = end_cofactor - start_cofactor
            cofactor_times.append((i, j, cofactor_time))
            
            print(f"  Cofactor [{i}, {j}] = {cofactor}, time: {cofactor_time:.6f} seconds")
    
    end_cofactor_matrix = time.time()
    cofactor_matrix_time = end_cofactor_matrix - start_cofactor_matrix
    
    print(f"\nCofactor matrix calculation time: {cofactor_matrix_time:.6f} seconds")
    print("Cofactor Matrix:")
    print(cofactor_matrix)
    
    # Calculate the adjugate (transpose of cofactor matrix)
    print("\nStep 3: Calculate the adjugate (transpose of cofactor matrix)")
    start_adjugate = time.time()
    adjugate = cofactor_matrix.transpose()
    end_adjugate = time.time()
    adjugate_time = end_adjugate - start_adjugate
    
    print(f"  Adjugate calculation time: {adjugate_time:.6f} seconds")
    print("Adjugate Matrix:")
    print(adjugate)
    
    # Calculate the inverse
    print("\nStep 4: Calculate the inverse (adjugate / determinant)")
    start_inverse = time.time()
    inverse = adjugate / det
    end_inverse = time.time()
    inverse_time = end_inverse - start_inverse
    
    print(f"  Inverse calculation time: {inverse_time:.6f} seconds")
    print("Inverse Matrix:")
    print(inverse)
    
    end_total = time.time()
    total_time = end_total - start_total
    
    print("\nTiming Summary:")
    print(f"  Determinant calculation time: {det_time:.6f} seconds")
    print(f"  Cofactor matrix calculation time: {cofactor_matrix_time:.6f} seconds")
    print(f"  Adjugate calculation time: {adjugate_time:.6f} seconds")
    print(f"  Inverse calculation time: {inverse_time:.6f} seconds")
    print(f"  Total calculation time: {total_time:.6f} seconds")
    
    return inverse



M_inv = calculate_inverse_4x4_with_timing(M_simp)

Step 1: Calculate the determinant
Computing 4x4 determinant:
  Total determinant: ((-m**2*r**2*(sqrt(2)*h*sin(q4 + pi/4) + h2*cos(q4))**2 + (Iaz + h**2*m + m*r**2)*(-Iax*sin(q4)**2 + Iax + Iay*sin(q4)**2 + Isx + h**2*m + h**2 + 2*h*h2*m + h2**2*m + m*r**2*sin(q4)**2))*(-Iax*(sin(pit)*cos(q4) - sin(q4)*sin(rol)*cos(pit))*sin(q4)*cos(rol) + Iay*(sin(pit)*sin(q4) + sin(rol)*cos(pit)*cos(q4))*cos(q4)*cos(rol) + Iaz*(sin(pit - 2*rol) - sin(pit + 2*rol))/4 - Isy*(sin(pit - 2*rol) - sin(pit + 2*rol))/4 + Isz*(sin(pit - 2*rol) - sin(pit + 2*rol))/4 - h**2*(sin(pit)*sin(yaw)*cos(rol) - sin(rol)*cos(yaw))*cos(pit)*cos(rol)*cos(yaw) + h**2*(sin(pit)*cos(rol)*cos(yaw) + sin(rol)*sin(yaw))*sin(yaw)*cos(pit)*cos(rol) + m*(h*sin(rol) + h2*sin(rol) - r*sin(q4)*cos(rol))*(h*cos(pit)*cos(rol) + h2*cos(pit)*cos(rol) - r*sin(pit)*cos(q4) + r*sin(q4)*sin(rol)*cos(pit))) - (m*r*(sqrt(2)*h*sin(q4 + pi/4) + h2*cos(q4))*(Iaz*cos(pit)*cos(rol) - m*r*(-sqrt(2)*h*sin(pit)*sin(q4 + pi/4) - sqrt(2)*h*sin(rol)*cos(p

In [40]:
pts[1]

Matrix([
[h*sin(pit)*cos(rol)*cos(yaw) + h*sin(rol)*sin(yaw)],
[h*sin(pit)*sin(yaw)*cos(rol) - h*sin(rol)*cos(yaw)],
[                               h*cos(pit)*cos(rol)],
[                                                 1]])

In [47]:
pts_sym = []
for i in range(len(pts)):
    pts_sym.append(sp.MatrixSymbol(f"pts_{i}", 3, 1))

pts_sym = sp.MatrixSymbol(f"pts", len(pts), 3)
pts_rhs = Matrix([pts[i][0:3] for i in range(len(pts))])

codeblock = CodeBlock(
    Assignment(pts_sym, pts_rhs),
).cse() #common subexpression elimination
num_ops = sp.count_ops(codeblock)
code = sp.ccode(codeblock)

code_rep = code
for n in range(3*len(pts)):
    row = n // 3
    col = n % 3
    code_rep = code_rep.replace(f"[{n}]", f"[{row},{col}]")
code_rep = code_rep.replace(";", "")

print(f"# {num_ops} operations")
print(code_rep)

# 124 operations
x0 = sin(rol)
x1 = sin(yaw)
x2 = x0*x1
x3 = h*x2
x4 = sin(pit)
x5 = cos(rol)
x6 = cos(yaw)
x7 = x5*x6
x8 = x4*x7
x9 = h*x8
x10 = x3 + x9
x11 = x0*x6
x12 = h*x11
x13 = x1*x5
x14 = x13*x4
x15 = h*x14
x16 = -x12 + x15
x17 = cos(pit)
x18 = x17*x5
x19 = h*x18
x20 = r*x17
x21 = 0.29999999999999999*x20
x22 = r*x4
x23 = 1.0/(m + 1)
x24 = h2*m
x25 = cos(q4)
x26 = x20*x25
x27 = x26*x6
x28 = sin(q4)
x29 = r*x28
x30 = m*x29
x31 = m*x22*x28
x32 = x1*x26
x33 = h2*x18
x34 = x22*x25
x35 = x0*x17*x29
pts[0,0] = 0
pts[0,1] = 0
pts[0,2] = 0
pts[1,0] = x10
pts[1,1] = x16
pts[1,2] = x19
pts[2,0] = x10 + x21*x6
pts[2,1] = x1*x21 + x16
pts[2,2] = x19 - 0.29999999999999999*x22
pts[3,0] = x10
pts[3,1] = x16
pts[3,2] = x19
pts[4,0] = x23*(m*x27 + m*x3 + m*x9 + x10 + x11*x31 - x13*x30 + x2*x24 + x24*x8)
pts[4,1] = x23*(-m*x12 + m*x15 + m*x32 - x11*x24 + x14*x24 + x16 + x2*x31 + x30*x7)
pts[4,2] = x23*(m*x19 + m*x33 - m*x34 + m*x35 + x19)
pts[5,0] = h2*(x2 + x8) + x10 + x27 + x29*(x11*x4 - x13)
p

In [30]:
CodeBlock(
    [Assignment(pts_sym[i], Matrix(pts[i][0:3])) for i in range(len(pts))],
)

CodeBlock(
[Assignment(pts_0, Matrix([
[0],
[0],
[0]])), Assignment(pts_1, Matrix([
[h*sin(pit)*cos(rol)*cos(yaw) + h*sin(rol)*sin(yaw)],
[h*sin(pit)*sin(yaw)*cos(rol) - h*sin(rol)*cos(yaw)],
[                               h*cos(pit)*cos(rol)]])), Assignment(pts_2, Matrix([
[h*sin(pit)*cos(rol)*cos(yaw) + h*sin(rol)*sin(yaw) + 0.3*r*cos(pit)*cos(yaw)],
[h*sin(pit)*sin(yaw)*cos(rol) - h*sin(rol)*cos(yaw) + 0.3*r*sin(yaw)*cos(pit)],
[                                        h*cos(pit)*cos(rol) - 0.3*r*sin(pit)]])), Assignment(pts_3, Matrix([
[h*sin(pit)*cos(rol)*cos(yaw) + h*sin(rol)*sin(yaw)],
[h*sin(pit)*sin(yaw)*cos(rol) - h*sin(rol)*cos(yaw)],
[                               h*cos(pit)*cos(rol)]])), Assignment(pts_4, Matrix([
[(h*m*sin(pit)*cos(rol)*cos(yaw) + h*m*sin(rol)*sin(yaw) + h*sin(pit)*cos(rol)*cos(yaw) + h*sin(rol)*sin(yaw) + h2*m*sin(pit)*cos(rol)*cos(yaw) + h2*m*sin(rol)*sin(yaw) + m*r*sin(pit)*sin(q4)*sin(rol)*cos(yaw) - m*r*sin(q4)*sin(yaw)*cos(rol) + m*r*cos(pit)*cos(q

In [39]:
testsym = sp.MatrixSymbol("test", 1, 4)
codeblock = CodeBlock(

    Assignment(testsym, G),
)
codeblock

CodeBlock(
Assignment(test, Matrix([[0, -g*h*sin(pit)*cos(rol) + g*m*(-h*sin(pit)*cos(rol) - h2*sin(pit)*cos(rol) - r*sin(pit)*sin(q4)*sin(rol) - r*cos(pit)*cos(q4)), -g*h*sin(rol)*cos(pit) + g*m*(-h*sin(rol)*cos(pit) - h2*sin(rol)*cos(pit) + r*sin(q4)*cos(pit)*cos(rol)), g*m*(r*sin(pit)*sin(q4) + r*sin(rol)*cos(pit)*cos(q4))]]))
)